In [1]:
import scipy.signal as signal
import numpy as np
import matplotlib.pyplot as plt
import wfdb
import os

In [2]:
def haar_filter(x,fs):
   # Compute B1 and B2
    B1 = int(0.025 * fs)  # 9 for fs = 360
    B2 = int(0.06 * fs)   # 21 for fs = 360

    # Define c to ensure sum of h(n) = 0
    c = 2 * (B2 - B1) / (2 * B1 + 1)

    # Define the range of h(n)
    n = np.arange(-B2, B2 + 1)

    # Define h(n) based on the conditions
    h = np.zeros_like(n, dtype=float)
    h[np.abs(n) <= B1] = c
    h[(np.abs(n) > B1) & (np.abs(n) <= B2)] = -1

    return signal.convolve(x,h,mode='same')

    #this also works, however even slower
    y1 = np.zeros(len(x))
    y2= np.zeros(len(x))
    y = np.zeros(len(x))
    tmp= np.zeros(B2)
    x_padded = np.concatenate((tmp,x,tmp))
    #go through samples

    tmp = np.zeros(2*(B1+B2))
    tmp[:B2] = c
    tmp[B2:B1+B2] = -1

    for i in range(len(y)):
        #plus B2 in x[] is because of padding
        y1[i] = (c+1)*(x_padded[i+B1 + B2] - x_padded[i-B1 +B2]) + y1[i-1]
        y2[i] = -(x_padded[i+B2 + B2] - x_padded[i-B2 + B2] + y2[i-1])
        y[i] = y1[i] + y2[i]
        #y[i] = -sum(x_padded[i-B1+B2:i+B1+B2+1]) + (c+1) * sum(x_padded[i-B2+B2:i+B2+B2+1])
    return y

In [3]:
def get_candidates(y,x_bsl, fs, c1 = 0.55):
    x2 = np.zeros(len(x_bsl))
    #fill x2 by the formula from paper
    x2[1:len(x_bsl)-1] = np.array( [2 * x_bsl[n] - x_bsl[n+1] - x_bsl[n-1] for n in range(1, len(x_bsl)-1)] )
    
    #calculate s
    s = np.array([y[n]*(x_bsl[n] + c1*x2[n]) for n in range(len(y))])
    
    #tmp just to find the correct range of checking
    tmp = int(0.2*fs)
    
    #find candidates
    candidates =[]
    for k in range(tmp, len(s)-tmp):
        for n in range(-tmp, tmp):
            if np.abs(s[k]) < np.abs(s[k+n]):
                break
            elif n == tmp-1:
                candidates.append(k)

    return candidates, s

In [4]:
#še popravki - skippam če jih ni 5 še prepoznanih (za W1 in W2) - da se bolš nrdit


def detect_beats(s,x_bsl,candidates, fs, T=0.1, beta1=0.5, beta2=0.5, taus=[0.08,  0.12, 0.2, 0.25, 0.35], omega_treshold = 0.1):
    #current s in the window
    s_current = []
    #window size
    seconds_window = 10
    samples_window = fs*seconds_window
    #recent detected number
    recent_detected_num = 5
    #detected beats
    detected = []

    #for every candidate
    for i in candidates:
        s_current = []
        for j in range(i-samples_window, i):
            #skip if out of range
            if j < 0:
                continue
            s_current.append(abs(s[j]))

        #calculate W1, get 5th largest value in the last 10 seconds
        W1 = T
        if len(s_current) >= 5:
            W1 = sorted(s_current)[-5] +T

            
        #calculate W2
        W2 = beta1
        det_len = len(detected)
        #only if there is at least 5 detected beat 
        if det_len > 5:
            recent_det = detected[det_len - recent_detected_num: det_len]

            mew = np.mean([recent_det[i] - recent_det[i+1] for i in range(3)])



            if recent_det[4] - recent_det[3] < 0.7*mew and det_len > 6:
                tmp = recent_det
                recent_det = [detected[-6]]
                recent_det.extend(tmp)
                recent_det = recent_det[:5]

                Ie = sum([taus[i] * (recent_det[i] - recent_det[i+1]) for i in range(4)])
                W2 = beta1 + beta2 * abs((i - recent_det[4])/(2*Ie) - round((i - recent_det[4])/(2*Ie)))
                print(W2)
        
            else:
                Ie = sum([taus[i] * (recent_det[i] - recent_det[i+1]) for i in range(4)])
                W2 = beta1 + beta2 * abs((i - recent_det[4])/Ie - round((i - recent_det[4])/Ie))
            
        
        treshold = W1*W2

        #variation ratio
        omega = 1/2

        if i - int(0.1*fs) > 0:
            tmp_list=  [x_bsl[j] for j in range(i - int(0.1*fs), i + int(0.1*fs))]
            u1 = max(tmp_list) - min(tmp_list)
            u2 = sum([abs(x_bsl[j] - x_bsl[j-1]) for j in range(i - int(0.1*fs)+1, i + int(0.1*fs) +1)])
            omega = u1/u2

      
        if abs(s[i]) > treshold:  
            if omega > omega_treshold:
                detected.append(i)

        #else :
            #print(s[i], treshold, i)

    return detected


In [10]:
def evaluate_results(annotation_beats, detected_beats,fs):
    time_tolerance = 0.15/2
    tolerance = round(time_tolerance*fs)

    not_detected = []
    for i in annotation_beats:
        for j in range(i-tolerance, i+tolerance+1):
            if j in detected_beats:
                break
        if j == i+tolerance:
            not_detected.append(i)
            #print(i)


    falsely_detected = []
    for i in detected_beats:
        for j in range(i-tolerance, i+tolerance+1):
            if j in annotation_beats:
                break
        if j == i+tolerance:
            falsely_detected.append(i)
            
    #print('Not detected:', len(not_detected))
    #print('Falsely detected:', len(falsely_detected))

    SE = (len(detected_beats) - len(falsely_detected))/(len(annotation_beats)+10e-8)
    PP = (len(detected_beats) - len(falsely_detected))/(len(detected_beats) +10e-8)
    print('Sensitivity:', SE)
    print('Positive predictivity:', PP)
    #print("not det " + str(not_detected))
    #print("false detected " + str(falsely_detected))
    
    return SE,PP, not_detected, falsely_detected

In [11]:
def eval_one_rec(database, record_num,T=0.1, beta2=0.1, beta1=0.1, taus=[0.08,  0.12, 0.2, 0.25, 0.35], omega_treshold = 0.1):
    #get only number of file
    record_num = str(record_num)
    sig = wfdb.rdsamp(database+ "/" + record_num)
    x_both = sig[0][:].flatten()
    x = x_both[0::2]

    #read sampling frequency
    fs = wfdb.rdsamp(database + "/"+ record_num)[1]['fs']
    #read annotation file
    record_name = database+ "/" + record_num
    annotation = wfdb.rdann(record_name, 'atr')
    beat_peaks = annotation.sample[:]
    #get rid of baseline
    butter_filter = signal.butter(5, 2, 'high', fs=fs, output='sos')
    x_bsl = signal.sosfilt(butter_filter, x)
    x_bsl[::-1] = signal.sosfilt(butter_filter, x_bsl[::-1])

    #delay
    delay = 1
    x_bsl[:-delay] = x_bsl[delay:]

    #apply haar filter
    y = haar_filter(x_bsl,fs)

    #get candidates
    candidates,s = get_candidates(y,x_bsl,fs)

    #detect beats
    detected = detect_beats(s,x_bsl,candidates, fs, T=T, beta2=beta2, beta1=beta1)

    #eval results
    print(record_num)
    SE,PP, not_detected, falsely_detected = evaluate_results(beat_peaks, detected,fs)
    
    return SE,PP, not_detected, falsely_detected, len(beat_peaks)

In [12]:
def eval_on_db(database, T=0.1, beta2=0.1, beta1=0.1, num_recordings=200):
    lst_SE_PP_lenOfSig = []

    files = [f for f in os.listdir(database) if os.path.isfile(os.path.join(database, f))]

    target_ext = ".hea"

    for file in files:
        if target_ext in file:
            record_num = file.split(".")[0]
            SE,PP, not_detected, falsely_detected, L = eval_one_rec(database,record_num, T=T, beta1=beta1, beta2=beta2)
            lst_SE_PP_lenOfSig.append([SE,PP,L])

            if len(lst_SE_PP_lenOfSig) > num_recordings:
                break

        else:
            continue

    mean_SE = np.mean([el[0] for el in lst_SE_PP_lenOfSig])
    mean_PP = np.mean([el[1] for el in lst_SE_PP_lenOfSig])
    whole_len = sum([el[2] for el in lst_SE_PP_lenOfSig])
    mean_SE_account_for_len = sum([el[0] * el[2]/whole_len for el in lst_SE_PP_lenOfSig])
    mean_PP_account_for_len = sum([el[1] * el[2]/whole_len for el in lst_SE_PP_lenOfSig])
    print("for beta1 = " + str(beta1) + ", beta2 = " + str(beta2) + ", T = " + str(T))
    print("mean SE: " + str(mean_SE))
    print("mean PP: " + str(mean_PP))
    print("SE accounting for length: " + str(mean_SE_account_for_len))
    print("PP accounting for length: " + str(mean_PP_account_for_len))

In [8]:
#grid search 
for T in [1]:
    for beta2 in [0.01,0.1,1]:
        for beta1 in [0.01,0.1,1]:
            if T==1 and beta2 == 0.01 and beta1 == 0.01:
                continue
            if T==1 and beta2 == 0.01 and beta1 == 0.1:
                continue
            eval_on_db("mit-bih-data", T=T, beta2=beta2, beta1=beta1)

for beta1 = 1, beta2 = 0.01, T = 1
mean SE: 0.11567677048547953
mean PP: 0.9985224071909693
SE accounting for length: 0.11194261719748191
PP accounting for length: 0.9983981464175923
for beta1 = 0.01, beta2 = 0.1, T = 1
mean SE: 0.9729947938125254
mean PP: 0.9080163839218881
SE accounting for length: 0.972773353887184
PP accounting for length: 0.9195421754582892
for beta1 = 0.1, beta2 = 0.1, T = 1
mean SE: 0.9567584361817526
mean PP: 0.9774290997409295
SE accounting for length: 0.9562438413398276
PP accounting for length: 0.982024609193744
for beta1 = 1, beta2 = 0.1, T = 1
mean SE: 0.08282131313759543
mean PP: 0.9982364394794595
SE accounting for length: 0.08029508109050805
PP accounting for length: 0.998050802070628
for beta1 = 0.01, beta2 = 1, T = 1
mean SE: 0.918257454998212
mean PP: 0.9913427447926916
SE accounting for length: 0.9140500856267132
PP accounting for length: 0.9923344333374067
for beta1 = 0.1, beta2 = 1, T = 1
mean SE: 0.8863341557921074
mean PP: 0.9964625614593391
SE 

In [13]:
#best parameters found out from the grid search
beta1 = 0.1
beta2 = 0.01
T = 1
eval_on_db("mit-bih-data", T=T, beta2=beta2, beta1=beta1)

100
Sensitivity: 0.9991204924802498
Positive predictivity: 0.999999999955986
101
Sensitivity: 0.9957310565103665
Positive predictivity: 0.9983948635099842
102
Sensitivity: 0.9922445255021787
Positive predictivity: 0.9945130315045948
103
Sensitivity: 0.9961740793402117
Positive predictivity: 0.9999999999519924
104
Sensitivity: 0.950237992170046
Positive predictivity: 0.9829901521493738
105
Sensitivity: 0.9502043849516832
Positive predictivity: 0.9729832571928089
106
Sensitivity: 0.9628217349398084
Positive predictivity: 0.9782082323981498
107
Sensitivity: 0.9892523364023714
Positive predictivity: 0.9810009267386006
108
Sensitivity: 0.9643640350348485
Positive predictivity: 0.9396367520865578
109
Sensitivity: 0.9984220906903976
Positive predictivity: 0.9999999999604899
111
Sensitivity: 0.9929676511489467
Positive predictivity: 0.9359257622211258
112
Sensitivity: 0.9956862744707574
Positive predictivity: 0.9984270546205889
113
Sensitivity: 0.9988864141982802
Positive predictivity: 0.50893